In [ ]:
import pickle
from sklearn.base import is_classifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# 📌 Stap 1: Lees de datasets in
datasets = {
    "student-mat": pd.read_csv("C:/Users/xande/Desktop/bit-academy/Portefolio Project/Portfolio-Project/Data/student-mat.csv", sep=";"),
    "student-por": pd.read_csv("C:/Users/xande/Desktop/bit-academy/Portefolio Project/Portfolio-Project/Data/student-por.csv", sep=";")
}

# Converteer de target naar een binaire classificatie
for name, data in datasets.items():
    data['G3_class'] = (data['G3'] >= 10).astype(bool)  # True/False classificatie

In [ ]:
# 📌 Stap 2: Preprocess data eerst! (X_train MOET beschikbaar zijn)
first_dataset = next(iter(datasets.values()))  # Kies een referentie dataset
first_dummies = pd.get_dummies(first_dataset, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(
    first_dummies.drop(columns=['G3', 'G3_class']),
    first_dummies['G3_class'],
    test_size=0.2,
    random_state=42
)

In [48]:
# 📌 Stap 3: Model opslag / inladen
class Readwritemodel():
    def __init__(self, model=None, filenaam=None):
        self.filenaam = filenaam
        self.model = model  
    
    def check_input(self, var):
        if isinstance(var, str) and var.endswith(".sav"):
            return True
        else:
            print("Jouw filenaam moet een string zijn en eindigen met .sav")
            return False

    def opslaanmodel(self):
        if self.check_input(self.filenaam):
            pickle.dump(self.model, open(self.filenaam, 'wb'))
            print(f"Jouw model is opgeslagen onder de naam: {self.filenaam}")
        
    def inladenmodel(self):
        if self.check_input(self.filenaam):
            try:
                geladen_model = pickle.load(open(self.filenaam, 'rb'))
                if is_classifier(geladen_model):
                    print("Jouw model is opgehaald en je kan er nu mee voorspellen!")
                    return geladen_model
                else:
                    print("Dit is geen classificatie model")
                    return None
            except FileNotFoundError:
                print("Het modelbestand kan niet gevonden worden. Er wordt een nieuw model getraind.")
                return None 


In [49]:
# 📌 Stap 4: Modelbeheer
func = Readwritemodel(filenaam="classificatie_model.sav")
model = func.inladenmodel()

if model is None:
    print("Ik train een model...")
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    func.model = model
    func.opslaanmodel()
else:
    print("Je gebruikt nu een opgeslagen model")

Jouw model is opgehaald en je kan er nu mee voorspellen!
Je gebruikt nu een opgeslagen model


In [50]:
# 📌 Stap 5: Loop door beide datasets en maak voorspellingen
for name, data in datasets.items():
    print(f"Verwerken van dataset: {name}")
    data_dummies = pd.get_dummies(data, drop_first=True)
    X = data_dummies.drop(columns=['G3', 'G3_class'], errors="ignore")
    
    # ✅ FIX: Controleer of `X_train` beschikbaar is
    if 'X_train' not in globals():
        raise NameError("⚠️ FOUT: X_train is niet gedefinieerd!")

    # 📌 Zorg dat kolommen overeenkomen
    missing_cols = set(X_train.columns) - set(X.columns)
    for col in missing_cols:
        X[col] = 0  # Voeg ontbrekende kolommen toe
    
    X = X[X_train.columns]  # Zorg dat X dezelfde kolomvolgorde heeft als de training set
    
    y_pred = model.predict(X)
    data['Predictions'] = y_pred
    
    output_filename = f"{name}_classification_predictions.csv"
    data.to_csv(output_filename, sep=";", index=False)
    print(f"Voorspellingen opgeslagen in {output_filename}")

Verwerken van dataset: student-mat
Voorspellingen opgeslagen in student-mat_classification_predictions.csv
Verwerken van dataset: student-por
Voorspellingen opgeslagen in student-por_classification_predictions.csv


In [51]:
# 📌 Stap 7: Evaluatie op de testset
accuracy = accuracy_score(y_test, model.predict(X_test))
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, model.predict(X_test)))
print("Classification Report:")
print(classification_report(y_test, model.predict(X_test)))


Accuracy: 0.9240506329113924
Confusion Matrix:
[[25  2]
 [ 4 48]]
Classification Report:
              precision    recall  f1-score   support

       False       0.86      0.93      0.89        27
        True       0.96      0.92      0.94        52

    accuracy                           0.92        79
   macro avg       0.91      0.92      0.92        79
weighted avg       0.93      0.92      0.92        79

